<h1>CURRENTS Mesh Network - cadCAD Model</h1>

<h2>Model introduction</h2>

Individual host KPIs


### Assumptions
 

### Constraints / Scope


# 0. Dependencies

In [158]:
# Standard libraries: https://docs.python.org/3/library/
import math
from numpy import random
import numpy as np

# Analysis and plotting modules
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from random import normalvariate

# cadCAD configuration modules
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment

# cadCAD simulation engine modules
from cadCAD.engine import ExecutionContext, Executor


# 1. State Variables & System Parameters

In [159]:
# Time horizon (Days)
time = 365


# System states (python dictionaries):
initial_state = {
    'clients': 5,
    'host_revenue': 0,
    'demand': 0,
    'host_revenue': 0,
    'host_profit': 0,
    'host_expenses': 0,
    'cumulative_profit': 0

}


# System parameters (python lists):

system_params = {
    'price': [0.3],
    'host_line_cost': [0.06],
    'host_capacity': [2000],
    'potential_clients': [100], 
    'operating_expenses': [10],
    'avg_client_allocation': [20],
    'platform_fee': [0.02],
    'client_acquisition_rate_coeff': [0.013]
        
}


MONTE_CARLO_RUNS = 1

#flexible way to create unique seeds for each monte carlo run
seeds = [random.RandomState(i) for i in range(MONTE_CARLO_RUNS)] 

# 2. Policy functions

In [160]:
# Returns a dictionary
# Never updates a state directly. Rather, it returns an Input/Signal which is used by a State Update function to update the state



def p_client_acquisition(params, substep, state_history, previous_state):
    
    run = previous_state['run']
    
    hostCapacity = params['host_capacity']
    clientAllocation = params['avg_client_allocation']
    
    maxClients = hostCapacity/clientAllocation
    
    if(maxClients > previous_state['clients']):
    
        # S-shaped growth representing client acquisition ('diffusion of innovation')
        x = state_history[-1][-1]['timestep']
        mp = time/2 # midpoint of sigmoid
        #k = 0.015 # steepness of curve (default)
        k = params['client_acquisition_rate_coeff']
        height = params['potential_clients']

        clients = height / (1+ math.e**(-k*(x-mp))) 
        clients = int(clients)
        
    else:
        clients = maxClients
    
    return {'newClients': clients}


# calculate daily demand 
def p_demand(params, substep, state_history, previous_state):
    
    demand = previous_state['clients'] * params['avg_client_allocation']
    
    return {'demand': demand}



def p_revenue(params, substep, state_history, previous_state):
    
    revenue = previous_state['demand'] * params['price']
    
    return {'host_revenue': revenue}



def p_expenses(params, substep, state_history, previous_state):
    
    platformCommission = previous_state['host_revenue'] * params['platform_fee']
    
    expenses = (params['host_line_cost']*params['host_capacity']) + params['operating_expenses'] - platformCommission
    
    return {'host_expenses': expenses}



def p_profit(params, substep, state_history, previous_state):
    
    profit = previous_state['host_revenue'] - previous_state['host_expenses']
    
    return {'host_profit': profit}

def p_cumulative_profit(params, substep, state_history, previous_state):
    
    cumulativeProfit = previous_state['cumulative_profit'] + previous_state['host_profit']
    
    return {'cumulative_profit': cumulativeProfit}




# 3. State Update Functions

In [161]:
# Update number of clients based on s-curve
def s_clients(params, substep, state_history, previous_state, policy_input):
    
    x = policy_input['newClients'] 
    
    return ('clients', x)



# Update daily demand (i.e. bandwidth allocation)
def s_demand(params, substep, state_history, previous_state, policy_input):
    
    x = policy_input['demand']
    
    return ('demand', x)



# Update daily revenue
def s_revenue(params, substep, state_history, previous_state, policy_input): 
    
    x = policy_input['host_revenue']
    
    return ('host_revenue', x)



# Update daily expenses
def s_expenses(params, substep, state_history, previous_state, policy_input):  
    
    x = policy_input['host_expenses']
    
    return ('host_expenses', x)



# Update daily profit
def s_profit(params, substep, state_history, previous_state, policy_input):  
    
    x = policy_input['host_profit'];
    
    return ('host_profit', x)

# Update cumulative profit
def s_cumulative_profit(params, substep, state_history, previous_state, policy_input):  
    
    x = policy_input['cumulative_profit'];
    
    return ('cumulative_profit', x)






# 4. Partial State Update Blocks

In [162]:
partial_state_update_blocks = [
    {
        'policies': {
            'client_acquisition': p_client_acquisition,
            'demand': p_demand,
            'revenue': p_revenue,
            'expenses': p_expenses,
            'profit': p_profit,
            'cumulative_profit': p_cumulative_profit
        },
        'variables': {
            'active_clients': s_clients,
            'demand': s_demand,
            'revenue': s_revenue,
            'expenses': s_expenses,
            'profit': s_profit,
            'cumulative_profit': s_cumulative_profit
        }
    }
]

# 5. Simulation Execution

In [163]:
from cadCAD import configs
del configs[:] # Clear any prior configs


sim_config = config_sim({
    'N': 1,
    'T': range(time),
    'M': system_params
})


experiment = Experiment()
experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)



exec_context = ExecutionContext()
run = Executor(exec_context=exec_context, configs=configs)

(system_events, tensor_field, sessions) = run.execute()


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (365, 8, 1, 6)
Execution Method: local_simulations
SimIDs   : [0]
SubsetIDs: [0]
Ns       : [0]
ExpIDs   : [0]
Execution Mode: single_threaded
Total execution time: 0.03s


# 6. Simulation Output Preparation

### Check initial data

In [164]:
simulation_result = pd.DataFrame(system_events)
simulation_result.head(100)

clients  host_revenue  demand  host_profit  host_expenses  \
0         5           0.0       0         0.00           0.00   
1         8           0.0     100         0.00         130.00   
2         8          30.0     160      -130.00         130.00   
3         8          48.0     160      -100.00         129.40   
4         8          48.0     160       -81.40         129.04   
..      ...           ...     ...          ...            ...   
95       24         138.0     460        10.76         127.24   
96       24         138.0     480        10.76         127.24   
97       24         144.0     480        10.76         127.24   
98       24         144.0     480        16.76         127.12   
99       25         144.0     480        16.88         127.12   

    cumulative_profit  simulation  subset  run  substep  timestep  
0                0.00           0       0    1        0         0  
1                0.00           0       0    1        1         1  
2                0.00           0       0    1        1         2  
3             -130.00           0       0    1        1         3  
4             -230.00           0       0    1        1         4  
..                ...         ...     ...  ...      ...       ...  
95           -4069.44           0       0    1        1        95  
96           -4058.68           0       0    1        1        96  
97           -4047.92           0       0    1        1        97  
98           -4037.16           0       0    1        1        98  
99           -4020.40           0       0    1        1        99  

[100 rows x 11 columns]

### Clean substeps

In [165]:


# Convert system events dictionary into a pandas dataframe
# Get system events and attribute index
df = (pd.DataFrame(system_events)
        .assign(days=lambda df: df.timestep)
        .query('timestep > 1')
     )

# Clean substeps
first_ind = (df.substep == 0) & (df.timestep == 0)
last_ind = df.substep == max(df.substep)
inds_to_drop = (first_ind | last_ind)
df = df.loc[inds_to_drop].drop(columns=['substep'])

# Attribute parameters to each row
df = df.assign(**configs[0].sim_config['M'])
for i, (_, n_df) in enumerate(df.groupby(['simulation', 'subset', 'run'])):
    df.loc[n_df.index] = n_df.assign(**configs[i].sim_config['M'])
    

# 7. Plot data

In [166]:
pd.options.plotting.backend = "plotly"

# plot some base scenario
#fig_df = df.query('avg_client_allocation == 10')
fig_df = df

fig_df.plot(
    kind='line',
    x='timestep',
    y=['clients'],
    height=500,
    animation_frame='client_acquisition_rate_coeff'
)

In [167]:
fig = px.line(
    fig_df,
    x='timestep',
    y=['host_revenue', 'demand'],
    height=500,
    animation_frame='run'
)

fig.show()

In [168]:
fig = px.line(
    fig_df,
    x='timestep',
    y=['host_revenue', 'host_expenses', 'host_profit'],
    height=500,
    animation_frame='run'
)

fig.show()

In [170]:
fig = px.line(
    fig_df,
    x='timestep',
    y='cumulative_profit',
    height=500,
    animation_frame='run'
)

fig.show()

## Get insights directly from System Events

#### Given the parameters, when does the host become profitable?

In [185]:
for timestep in system_events:
    if(timestep['cumulative_profit'] > 0):
        print('The host becomes profitable after:\n', timestep['timestep'], 'days.')
        break


The host becomes profitable after:
 161 days.
